In [ ]:
# Usando dados como pontos marcados, sofridos, eficiência ofensiva/defensiva e rebotes, prever quantas vitórias uma equipa terá numa temporada

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np


# 1️ Ler dados originais (sem tocar neles)
teams_df = pd.read_csv("data/teams.csv")

# 2️ Criar uma cópia para manipulação
teams_model = teams_df.copy()

# 3️ Criar features derivadas úteis para prever vitórias
teams_model["point_diff"] = teams_model["o_pts"] - teams_model["d_pts"]
teams_model["off_efficiency"] = teams_model["o_pts"] / teams_model["o_fga"]
teams_model["def_efficiency"] = teams_model["d_pts"] / teams_model["d_fga"]
teams_model["rebalance"] = teams_model["o_reb"] - teams_model["d_reb"]

#Remove linhas com valores nulos em colunas importantes.
teams_model = teams_model.dropna(subset=["won", "o_pts", "d_pts"])


teams_model.to_csv("data_models/teams_model.csv", index=False)

In [9]:
import pandas as pd
import numpy as np

# 1️⃣ Ler os dados das jogadoras por equipa e ano
players_teams = pd.read_csv("data/players_teams.csv")

# 2️⃣ Criar uma métrica de rating individual (como já fizeste)
players_teams["player_rating"] = (
    players_teams["points"]
    + players_teams["rebounds"]
    + 1.5 * players_teams["assists"]
    + 2 * players_teams["steals"]
    + 2 * players_teams["blocks"]
    - players_teams["turnovers"]
)

# 3️⃣ Selecionar apenas colunas relevantes
players_teams = players_teams[["playerID", "year", "tmID", "player_rating"]]

# 4️⃣ Criar coluna com o rating do ano anterior de cada jogadora
players_teams = players_teams.sort_values(["playerID", "year"])
players_teams["player_rating_prev"] = players_teams.groupby("playerID")["player_rating"].shift(1)

# 5️⃣ Agora, para cada equipa e ano, somar o rating *do ano anterior* dos jogadores que estão nessa equipa
team_prev_ratings = (
    players_teams.groupby(["year", "tmID"], as_index=False)["player_rating_prev"]
    .mean()
    .rename(columns={"player_rating_prev": "team_sum_player_prev_rating"})
)

# 6️⃣ Remover anos iniciais (sem dados anteriores)
team_prev_ratings = team_prev_ratings.dropna(subset=["team_sum_player_prev_rating"])

# 7️⃣ Guardar para análise posterior
import os
os.makedirs("data_models", exist_ok=True)
team_prev_ratings.to_csv("data_models/team_prev_ratings.csv", index=False)

print("\n✅ Criada tabela com força da equipa (baseada no ano anterior):")
print(team_prev_ratings.head(10))



✅ Criada tabela com força da equipa (baseada no ano anterior):
    year tmID  team_sum_player_prev_rating
13     1  SEA                     8.250000
14     1  UTA                   160.500000
16     2  CHA                   408.222222
17     2  CLE                   302.846154
18     2  DET                   454.857143
19     2  HOU                   311.357143
20     2  IND                   373.300000
21     2  LAS                   572.944444
22     2  MIA                   361.875000
23     2  MIN                   369.357143
